In [1]:
import torch
import torch.nn as nn
import math

https://www.youtube.com/watch?v=MswxJw-8PvE

https://pytorch.org/docs/stable/notes/extending.html#extending-autograd

# autograd

https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/

https://pytorch.org/docs/stable/autograd.html

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

https://pytorch.org/docs/stable/notes/autograd.html

# torch.autograd.Function

https://pytorch.org/docs/stable/autograd.html#function

https://pytorch.org/tutorials/beginner/examples_autograd/polynomial_custom_function.html#sphx-glr-beginner-examples-autograd-polynomial-custom-function-py

https://hongl.tistory.com/206

All functions(mathematical operations) applied to tensors to construct computational graph are an object of torch.autograd.Function class.

Any subclasses of this class  forward() function

# Double Backward
https://pytorch.org/tutorials/intermediate/custom_function_double_backward_tutorial.html#

In [55]:
x = torch.randn(3,4); w1 = torch.randn(4,2, requires_grad=True)
y = torch.randn(3,4); w2 = torch.randn(4,2, requires_grad=True)

out = x.matmul(w1) + y.matmul(w2)
l = out.sum()

print(out.grad_fn)
print(out.grad_fn.next_functions)
print(out.grad_fn.next_functions[0][0].next_functions)
print(out.grad_fn.next_functions[1][0].next_functions)

((<MmBackward0 object at 0x7f71135145e0>, 0), (<MmBackward0 object at 0x7f7113935af0>, 0))
((None, 0), (<AccumulateGrad object at 0x7f71138e54f0>, 0))
((None, 0), (<AccumulateGrad object at 0x7f711381aa00>, 0))


In [60]:
print(l.grad_fn)
print(l.grad_fn.next_functions)
print(l.grad_fn.next_functions[0][0].next_functions)

((<AddBackward0 object at 0x7f7113608310>, 0),)
((<MmBackward0 object at 0x7f71137735b0>, 0), (<MmBackward0 object at 0x7f7113773eb0>, 0))


In [66]:
x = torch.randn(3,4); w1 = torch.randn(4,2, requires_grad=True)
y = torch.randn(3,4); w2 = torch.randn(4,2, requires_grad=True)
z = torch.randn(3,4); w3 = torch.randn(4,2, requires_grad=True)

out = x.matmul(w1) + y.matmul(w2) + z.matmul(w3)
l = out.sum()

print(out.grad_fn)
print(out.grad_fn.next_functions)
print(out.grad_fn.next_functions[0][0].next_functions)
print(out.grad_fn.next_functions[1][0].next_functions)

((<AddBackward0 object at 0x7f7113865ee0>, 0), (<MmBackward0 object at 0x7f71135edeb0>, 0))
((<MmBackward0 object at 0x7f7156f5deb0>, 0), (<MmBackward0 object at 0x7f71135edeb0>, 0))
((None, 0), (<AccumulateGrad object at 0x7f71134f7c70>, 0))


In [69]:
print(l.grad_fn)
print(l.grad_fn.next_functions)
print(l.grad_fn.next_functions[0][0].next_functions)
print(l.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)

((<AddBackward0 object at 0x7f711573b700>, 0),)
((<AddBackward0 object at 0x7f711573b6d0>, 0), (<MmBackward0 object at 0x7f711637f310>, 0))
((<MmBackward0 object at 0x7f711573b820>, 0), (<MmBackward0 object at 0x7f711573b700>, 0))


# grad_fn

https://amsword.medium.com/understanding-pytorchs-autograd-with-grad-fn-and-next-functions-b2c4836daa00

## Example 1

In [2]:
# make data

x = torch.linspace(0., 2 * math.pi, steps=10, requires_grad=True)
print('x:')
print(x, '\n')

# 1
print('w = sin(x):')
w = torch.sin(x)
print(w, '\n')    # has grad_fn

# 2
print('y = 2 * w:')
y = 2 * w
print(y, '\n')

# 3
print('z = y + 1:')
z = y + 1
print(z, '\n')

# 4
print('l = sum(z):')
l = z.sum()
print(l, '\n')

x:
tensor([0.0000, 0.6981, 1.3963, 2.0944, 2.7925, 3.4907, 4.1888, 4.8869, 5.5851,
        6.2832], requires_grad=True) 

w = sin(x):
tensor([ 0.0000e+00,  6.4279e-01,  9.8481e-01,  8.6603e-01,  3.4202e-01,
        -3.4202e-01, -8.6603e-01, -9.8481e-01, -6.4279e-01,  1.7485e-07],
       grad_fn=<SinBackward0>) 

y = 2 * w:
tensor([ 0.0000e+00,  1.2856e+00,  1.9696e+00,  1.7321e+00,  6.8404e-01,
        -6.8404e-01, -1.7321e+00, -1.9696e+00, -1.2856e+00,  3.4969e-07],
       grad_fn=<MulBackward0>) 

z = y + 1:
tensor([ 1.0000,  2.2856,  2.9696,  2.7321,  1.6840,  0.3160, -0.7321, -0.9696,
        -0.2856,  1.0000], grad_fn=<AddBackward0>) 

l = sum(z):
tensor(10.0000, grad_fn=<SumBackward0>) 



In [154]:
# gradient history of l

dl = torch.tensor(1.)   # dl/dl
print('dl/dl:', dl)

print('-' * 50)

print('l.grad_fn:', l.grad_fn)
dz = l.grad_fn(dl)      # dl/dz = dl/dz * dl/dl
print('dl/dz:', dz)

print('-' * 50)

print('z.grad_fn:', l.grad_fn.next_functions)
dy = z.grad_fn(dz)      # dl/dy = dz/dy * dl/dz * dl/dl
print('dl/dy:', dy)

print('-' * 50)

print('y.grad_fn:', l.grad_fn.next_functions[0][0].next_functions)
dw = y.grad_fn(dy[0])   # dl/dw = dy/dw * dz/dy * dl/dz * dl/dl
print('dl/dw:', dw)

print('-' * 50)

print('w.grad_fn:', l.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)
dx = w.grad_fn(dw[0])   # dl/dx = dw/dx * dy/dw * dz/dy * dl/dz * dl/dl
print('dl/dx:', dx)

dl/dl: tensor(1.)
--------------------------------------------------
l.grad_fn: <SumBackward0 object at 0x7fd2fae52670>
dl/dz: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
--------------------------------------------------
z.grad_fn: ((<AddBackward0 object at 0x7fd2fadf53d0>, 0),)
dl/dy: (tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), None)
--------------------------------------------------
y.grad_fn: ((<MulBackward0 object at 0x7fd2fadf53d0>, 0), (None, 0))
dl/dw: (tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]), None)
--------------------------------------------------
w.grad_fn: ((<SinBackward0 object at 0x7fd2ff0b2fd0>, 0), (None, 0))
dl/dx: tensor([ 2.0000,  1.5321,  0.3473, -1.0000, -1.8794, -1.8794, -1.0000,  0.3473,
         1.5321,  2.0000], grad_fn=<MulBackward0>)


In [151]:
l.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].next_functions

((<AccumulateGrad at 0x7fd2faf69ee0>, 0),)

In [150]:
l.backward()
print('dl/dx:', x.grad)     # same gradient

dl/dx: tensor([ 2.0000,  1.5321,  0.3473, -1.0000, -1.8794, -1.8794, -1.0000,  0.3473,
         1.5321,  2.0000])


In [5]:
x.is_leaf

True